Import os and numpy libraries. Mount local Gdrive folder in Colabs for data access.

In [6]:
import os
import numpy as np  # load numpy
from google.colab import drive  # load google colab for mounting gdrive
drive.mount('/content/drive')  # mount gdrive to access emails


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Counter from collections. Import GaussianNB and accuracy_score from sklearn library.

In [7]:
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

Define a function that creates a list of all words (all_words) found in emails using iterators by email and then by line. Use the split function to divide lines into words to add to all_words. Next a dictionary is created using Counter() from all_words. Next non-alpha characters are removed and single characters are removed from the dictionary. Finally a list of the 3000 most common words are kept and the rest are discarded.

In [8]:
def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
  dictionary = dictionary.most_common(3000)
  return dictionary

Define a function that creates a word frequency matrix (702 X 3000). A frequency of the words is given for the top 3000 words by email.  Each of the 703 rows represents one email in the training data.  

In [9]:

def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels


Set up file sources using Gdrive path plus custom folder names.

In [10]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'

Here the 3000 word dictionary created using TRAIN_DIR email files.

In [11]:
dictionary = make_Dictionary(TRAIN_DIR)

Using a Gaussian statistics the likelyhood that an email is spam vs. non-spam is determined using the training emails. A model is created statistically that differentiates between spam and non-spam emails.

In [12]:
print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()
print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels)
print ("Training completed")

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed


Model created above is tested vs. sample known emails and accuarcy of model is calculated.

In [13]:


print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))

testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616
